In [4]:
!wget https://raw.githubusercontent.com/alexeygrigorev/ai-engineering-buildcamp-code/main/01-foundation/homework/books.csv

--2026-02-07 22:03:13--  https://raw.githubusercontent.com/alexeygrigorev/ai-engineering-buildcamp-code/main/01-foundation/homework/books.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 879 [text/plain]
Saving to: ‘books.csv’

books.csv           100%[===================>]     879  --.-KB/s    in 0s      

2026-02-07 22:03:13 (53.7 MB/s) - ‘books.csv’ saved [879/879]



In [5]:
!uv add 'markitdown[pdf]'
!sudo apt-get install -y poppler-utils

Resolved 179 packages in 1ms
Audited 174 packages in 4ms
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (24.02.0-1ubuntu9.8).
0 upgraded, 0 newly installed, 0 to remove and 88 not upgraded.


In [10]:
import pandas as pd
from pathlib import Path
import requests
import subprocess

# 1) Load the CSV
csv_url = "https://raw.githubusercontent.com/alexeygrigorev/ai-engineering-buildcamp-code/main/01-foundation/homework/books.csv"
df = pd.read_csv(csv_url)

# 2) Create directories
pdf_dir = Path("books")
md_dir = Path("books_text")
pdf_dir.mkdir(exist_ok=True)
md_dir.mkdir(exist_ok=True)

# 3) Download each PDF
for idx, row in df.iterrows():
    title = row["title"]
    pdf_url = row["pdf_url"]
    filename = pdf_url.split("/")[-1]
    pdf_path = pdf_dir / filename

    if not pdf_path.exists():
        print(f"Downloading {title}...")
        resp = requests.get(pdf_url)
        with open(pdf_path, "wb") as f:
            f.write(resp.content)

# 4) Convert to Markdown
for pdf_path in pdf_dir.glob("*.pdf"):
    md_path = md_dir / f"{pdf_path.stem}.md"
    print(f"Converting {pdf_path.name} → {md_path.name}...")
    with open(md_path, "w") as f:
        subprocess.run(
            ["markitdown", str(pdf_path)],
            stdout=f,
            check=True
        )

Converting Think-C.pdf → Think-C.md...
Converting thinkpython2.pdf → thinkpython2.md...
Converting PhysicalModelingInMatlab4.pdf → PhysicalModelingInMatlab4.md...
Converting thinkos.pdf → thinkos.md...
Converting thinkdsp.pdf → thinkdsp.md...
Converting thinkjava2.pdf → thinkjava2.md...
Converting thinkcomplexity2.pdf → thinkcomplexity2.md...


In [11]:
with open("books_text/thinkpython2.md") as f:
    print(sum(1 for _ in f))

16269
